In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from tqdm import tqdm
import scipy.sparse as sp

import warnings
warnings.simplefilter('ignore')

from numpy.linalg import svd

from rectools.dataset import Dataset
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k
from rectools.metrics.ranking import MAP
import scipy.sparse as scs

import os

In [2]:
%%time
us = os.getcwd()

if 'liliyaivannikova' in us:
    pass
elif 'Владислав' in us:
    PATH_imdb = 'C:/Users/Владислав/test_python_scripts/project/test project/dataset_imdb/'
    
#name_basics = pd.read_csv(PATH_imdb + 'name.basics.tsv', sep='\t') #список по людям, фио, профессия, года жизни, с какими фильмами связан
#title_crew = pd.read_csv(PATH_imdb + 'title.crew.tsv', sep='\t') # режиссер фильма
title_ratings = pd.read_csv(PATH_imdb + 'title.ratings.tsv', sep='\t') # рейтинг
#title_principals = pd.read_csv(PATH_imdb + 'title.principals.tsv', sep='\t')

CPU times: total: 453 ms
Wall time: 455 ms


In [3]:
%%time
us = os.getcwd()

if 'liliyaivannikova' in us:
    PATH = r'/Users/liliyaivannikova/Documents/project/ml-latest/'
elif 'Владислав' in us:
    PATH = 'C:/Users/Владислав/test_python_scripts/project/test project/dataset/'
    
movies = pd.read_csv(PATH + r'movies.csv')
#rating = pd.read_csv(PATH + r'ratings.csv')
links = pd.read_csv(PATH + r'links.csv')

CPU times: total: 78.1 ms
Wall time: 79 ms


### Препроцессинг
Тут есть заготовки подгрузки данных по режиссеру, сценаристу и актерам - но в текущей версии их не используем, т.к. это сильно раздует данные. Будем пробовать в DL части

In [4]:
'''
name_director = pd.merge(title_crew[['tconst','directors']],
                         name_basics[['nconst','primaryName']],
                         left_on='directors',
                         right_on='nconst',
                         how='left')\
    .drop(['directors','nconst'],axis=1)\
    .rename(columns={'primaryName': 'director'})

name_actors = pd.merge(title_principals[title_principals.category.isin(['actor','self'])][['tconst','nconst']],
                       name_basics[['nconst','primaryName']],
                       on='nconst',
                       how='left').drop(['nconst'],axis=1)

name_writer = pd.merge(title_principals[title_principals.category=='writer'][['tconst','nconst']],
                       name_basics[['nconst','primaryName']],
                       on='nconst',
                       how='left')\
    .drop(['nconst'],axis=1)\
    .rename(columns={'primaryName': 'writer'})
'''
title_ratings = title_ratings[['tconst','averageRating']].rename(columns={'averageRating': 'imdbRating'})

#name_director['tconst'] = name_director['tconst'].str.replace('tt', '').astype('Int64')
#name_actors['tconst'] = name_actors['tconst'].str.replace('tt', '').astype('Int64')
title_ratings['tconst'] = title_ratings['tconst'].str.replace('tt', '').astype('Int64')
#name_writer['tconst'] = name_writer['tconst'].str.replace('tt', '').astype('Int64')


movies_full_data = pd.merge(movies, links, on='movieId', how='left')\
    .drop('tmdbId',axis=1)\
.merge(title_ratings, left_on='imdbId', right_on='tconst', how='left')\
    .drop('tconst',axis=1)

#.merge(name_director, left_on='imdbId', right_on='tconst', how='left')\
#    .drop('tconst',axis=1)\
#.merge(name_writer, left_on='movieId', right_on='tconst', how='left')\
#    .drop('tconst',axis=1)\

movies_full_data = movies_full_data.drop('imdbId',axis=1)

###  Итоговая таблица

In [5]:
movies_full_data#[movies_full_data.imdbRating.isna()]

,movieId,title,genres,imdbRating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.3
1,2,Jumanji (1995),Adventure|Children|Fantasy,7.1
2,3,Grumpier Old Men (1995),Comedy|Romance,6.6
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,6.0
4,5,Father of the Bride Part II (1995),Comedy,6.1
...,...,...,...,...
86532,288967,State of Siege: Temple Attack (2021),Action|Drama,7.7
86533,288971,Ouija Japan (2021),Action|Horror,3.4
86534,288975,The Men Who Made the Movies: Howard Hawks (1973),Documentary,7.3
86535,288977,Skinford: Death Sentence (2023),Crime|Thriller,4.4


In [6]:
movies_full_data.to_csv('movies_full.csv', index=False)